<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: María Paz Raveau Morales</em><br>

</div>

# **<center>TAREA 1 MINERÍA DE TEXTO (NLP)</center>**
## <center>Extrayendo contenido útil y visualizando</center>
*20 de julio de 2025*

**Nombre Estudiante**: Cristian Tobar Morales  
**Asignatura**: Procesamiento de Lenguaje Natural 


## **01. INTRODUCCIÓN**

## **02. DESCRIPCIÓN DE LA BASE DE DATOS**

## **03. DESARROLLO**

### **CARGA Y ANÁLISIS EXPLORATORIO DE LOS DATOS**

#### Librerías requeridas

In [2]:
import pandas as pd
import numpy as np
import os

# Librerías de Procesamiento de Texto
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import stanza
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords


In [14]:
stanza.download('es') # Ejecutar solamente una vez
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma,depparse')

2025-07-20 11:53:10 INFO: Downloaded file to C:\Users\crist\stanza_resources\resources.json
2025-07-20 11:53:10 INFO: Downloading default packages for language: es (Spanish) ...
2025-07-20 11:53:12 INFO: File exists: C:\Users\crist\stanza_resources\es\default.zip
2025-07-20 11:53:17 INFO: Finished downloading models and saved to C:\Users\crist\stanza_resources
2025-07-20 11:53:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-07-20 11:53:18 INFO: Downloaded file to C:\Users\crist\stanza_resources\resources.json
2025-07-20 11:53:18 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-07-20 11:53:

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

#### Cargar e Información inicial del dataset

In [6]:
# Cargar los datos
df = pd.read_excel('data/resultadocabildoprovincial.xlsx')

In [7]:
# Dimensiones de los datos (Filas, columnas):
print("\u001b[1mDIMENSIONES\u001b[0m")
print("Filas:",df.shape[0]) # type: ignore
print("Columnas:",df.shape[1],"\n") # type: ignore

DIMENSIONES
Filas: 30414
Columnas: 12 



In [8]:
# Exploración inicial
df.head()

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento
0,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Autonomía/libertad,NaN,A,sin fundamento
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento
2,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Descentralización,NaN,A,La descentralizacion de ser autonoma por ser z...
3,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Equidad de género,NaN,A,Sin fundamento
4,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Respeto/conservación de la naturaleza o medio ...,NaN,A,Sin fundamento


In [9]:
# Información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30414 entries, 0 to 30413
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   region_nombre      30414 non-null  object
 1   provincia_nombre   30414 non-null  object
 2   comuna_nombre      30414 non-null  object
 3   cabildo_id         30414 non-null  int64 
 4   cabildo_nombre     30414 non-null  object
 5   cabildo_direccion  30414 non-null  object
 6   num_mesa           30414 non-null  int64 
 7   pregunta_glosa     30414 non-null  object
 8   concepto_nombre    30414 non-null  object
 9   concepto_otro      7121 non-null   object
 10  tipo-eleccion      30414 non-null  object
 11  fundamento         30414 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.8+ MB


Se observa que las columnas de interés `concepto_nombre` y `fundamento`, no presenta valores nulos.

concepto_nombre
Otro                                                                       7121
Igualdad                                                                    649
Justicia                                                                    619
A la salud                                                                  614
A la educación                                                              611
Descentralización                                                           610
Respeto/conservación de la naturaleza o medio ambiente                      607
Democracia                                                                  604
Plebiscitos, referendos y consultas                                         597
Protección, promoción y respeto de los derechos humanos y fundamentales     582
Respeto por la constitución                                                 567
Deberes de protección de conservación de la naturaleza                      565
Defensor del Pueblo/Ciud

### **PRE-PROCESAMIENTO DEL DATAFRAME**

Primeramente, se debe observar de la columna `fundamento`, cuales filas no aportar información relevante para el estudio, dicho caso se procede a eliminar.
Se crea un nueva columna `fundamento_modifcado`, con el objetivo de no alterar la columna original.

In [ ]:
columna = 'fundamento_modificado'

df[columna]= df['fundamento']


df[columna]=df.groupby([columna]).size().reset_index().sort_values(by=0,ascending=False)[0:10]

,fundamento,0
383,.,1655
14999,Sin fundamento,915
19011,sin fundamento,727
67,-,566
14067,SIN FUNDAMENTO,543
15004,Sin fundamento.,410
14936,Sin Fundamento,295
19015,sin fundamentos,214
15005,Sin fundamentos,206
15006,Sin fundamentos.,201


Para poder eliminar todas las filas `sin fundamento`, es crucial convertir todo el texto a minúsculas, debido a que Python es sensible a mayúsculas y minúsculas.

In [ ]:
# Transformar a minúscula
df[columna] = df[columna].str.lower()

# Eliminar las puntuaciones: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' 
df[columna] = df[columna].str.translate(str.maketrans('','', string.punctuation))

# Eliminar números
df[columna] = df[columna].str.translate(str.maketrans('', '', string.digits))

# Eliminar espacios en blanco
df[columna] = df[columna].str.strip()

# Reemplazar saltos de líneas por espacios
df[columna] = df[columna].str.replace('\n',' ')

In [ ]:
df = df[[len(x.split( )) >3  for x in df[columna]]]
df = df.reset_index(drop=True)

In [ ]:
df.groupby([columna]).size().reset_index().sort_values(by=0,ascending=False)[0:20]

In [ ]:
df[columna].sample(5).tolist()

#### Selección de concepto
Antes de seleccionar el concepto, de la columna `fundamento` se procederá eliminar las columnas que tienen texto duplicado, ya se las que dice "sin fundamento".

In [ ]:
# Se contara la cantidad
df['concepto_nombre'].value_counts().head(15)

['que exista la asamblea constituyente',
 'de acuerdo a los tiempos con la participación ciudadana',
 'se ratifica lo propuesto',
 'garantizar el derecho de expresión de todas las corrientes de opinión  eliminar cualquier forma de censura debe ser con responsabildiad y compromiso',
 'atenta a la libertad ya que está suscrito al respeto al otro']

,fundamento,0
272,acta no contiene fundamento,42
5143,el grupo sólo votó y no fundamentó,34
16070,sin descripción en fundamentos,26
5076,el grupo al tratar cada tema consideró estar d...,22
17502,validado por la comisión,20
15417,se entiende que el concepto abordado de forma ...,17
21,a favor en contra,17
6909,esta votación es la priorización de los encue...,17
10889,no se registran datos,17
1853,consagrar y garantizar este derecho fundamenta...,14


In [ ]:
print("\u001b[1mDIMENSIONES\u001b[0m")
cabildos.shape

(18530, 12)

El texto de la columna fundamento se tokeniza del siguiente modo: primero se convierte la columna en una lista de cadenas (**.to_list**); luego se unen todos los elementos en un único string separados por espacios (**.join**); a continuación se aplica **word_tokenize** al texto resultante; después se obtienen los tokens únicos mediante un set; y finalmente se devuelve la cantidad de dichos tokens únicos (**len**).

In [13]:
#tokenizar
len(set(word_tokenize(' '.join(cabildos['fundamento'].to_list()))))

13670

De la columna fundamento, el total de palabras que posee el texto es de 13670.

Lemmatizar

porque bueno lematizar al comienzo?
Es recomendable lemaitzar al comienzo porque entiende el lenguaje natural.

In [14]:
def lemmatizer(text):  
  doc = nlp(text)
  return ' '.join([x.lemma for x in doc.sentences[0].words])

Se crea un nueva columna llamada lemmatizado con el objetivo de trabajar con estos datos nuevos.
lematizar reduce la dimensionalidad

In [ ]:
cabildos['lemmatizado'] = [lemmatizer(x) for x in cabildos['fundamento']]